<a href="https://colab.research.google.com/github/benedettoscala/ifttt-code-generator/blob/main/fine_tuning_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece
!pip install -U bitsandbytes
!pip install rouge_score

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch
import pandas as pd
from datasets import Dataset


In [ ]:
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

# Preprocessing dei dati

In [ ]:
!git clone https://github.com/benedettoscala/ifttt-code-generator
%cd ifttt-code-generator/
!git pull

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import BitsAndBytesConfig

#Configurazione modello base e quantizzazione
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

#Caricamento del modello e tokenizer
print("Caricamento del modello base...")
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("Caricamento del tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Prepara il modello per k-bit training (disabilita gradienti su pesi int4 ecc.)
model = prepare_model_for_kbit_training(model)

# 3. Configurazione LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
    # i moduli dei transformer li ho controllati su hugging face, sono questi (godo)
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


In [ ]:
# Caricamento e pulizia del dataset
csv_path = "datasets/cleaned_and_combined.csv"
df = pd.read_csv(csv_path)

df.dropna(subset=["cleaned_description", "filter_code"], inplace=True)

#drop duplicates
df.drop_duplicates(subset=["cleaned_description", "filter_code"], inplace=True)

#Suddivisione train   e val
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset  = Dataset.from_pandas(eval_df)

In [ ]:
train_dataset

In [ ]:
from transformers import DataCollatorWithPadding
# Funzione di tokenizzazione
def tokenize_function(examples):
    separator = "\n###\n"

    # Concateno desc + code
    full_text = [
        desc + separator + code
        for desc, code in zip(examples["cleaned_description"], examples["filter_code"])
    ]

    # Tokenizza con padding e truncation "coerenti"
    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"  # cosi ottengo shape costanti
    )

    # Calcolo la lunghezza del prompt con gli stessi identici parametri
    prompt_text = [
        desc + separator
        for desc in examples["cleaned_description"]
    ]
    tokenized_prompt = tokenizer(
        prompt_text,
        truncation=True,
        max_length=512,
        padding="max_length"  # stesse impostazioni
    )
    prompt_lengths = [
        sum(p_id != tokenizer.pad_token_id for p_id in p_ids)
        for p_ids in tokenized_prompt["input_ids"]
    ]

    # Costruisco le label: maschero la parte del prompt con -100
    labels = []
    for i, seq in enumerate(tokenized["input_ids"]):
        prompt_len = prompt_lengths[i]
        # Il prompt è su N token e la rimanente parte su (512 - N) token
        # ATTENZIONE: se usi "padding=max_length" la seq avrà sempre lunghezza 512
        # e i token in eccesso (se esiste) saranno solo pad.
        # Esempio: seq[prompt_len:] serve a prendere la parte del "code".
        masked_labels = [-100]*prompt_len + seq[prompt_len:]
        labels.append(masked_labels)

    tokenized["labels"] = labels
    return tokenized




train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset  = eval_dataset.map(tokenize_function,  batched=True)

# Data collator con padding a sinistra
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding='longest'  # la dimensione massima del batch la decide a runtime
)

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
from transformers import Seq2SeqTrainingArguments, Trainer
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

# Impostazioni di training specifiche per Seq2Seq
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    learning_rate=1e-4,
    weight_decay=0.01,
    max_grad_norm=1.0,
    save_steps=100,
    logging_steps=2,
    eval_strategy="epoch",  # Puoi anche usare "epoch"
    load_best_model_at_end=False,
    save_total_limit=3,
    fp16=False,
    bf16=True,
    report_to="wandb",
    #predict_with_generate=True  # Abilitare la generazione delle predizioni
)

# Funzione per il calcolo delle metriche
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Verifica il tipo e il range dei token ID
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    # Assicurati che le predizioni siano in formato numpy array
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.detach().cpu().numpy()

    # Verifica se le predizioni contengono logits o token ID
    if predictions.dtype not in [int, 'int32', 'int64']:
        # Se sono logits, converti in token ID utilizzando argmax
        predictions = predictions.argmax(axis=-1)

    # Decodifica le predizioni e le etichette in testo
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU
    bleu_scores = [
        sentence_bleu([ref.split()], pred.split(), smoothing_function=SmoothingFunction().method1)
        for pred, ref in zip(decoded_preds, decoded_labels)
    ]
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    # METEOR
    # METEOR
    meteor_scores = [
        meteor_score([ref.split()], pred.split())
        for pred, ref in zip(decoded_preds, decoded_labels)
    ]

    avg_meteor = sum(meteor_scores) / len(meteor_scores)

    # ROUGE
    rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = [
        rouge.score(ref, pred)
        for pred, ref in zip(decoded_preds, decoded_labels)
    ]
    avg_rouge1 = sum(score["rouge1"].fmeasure for score in rouge_scores) / len(rouge_scores)
    avg_rouge2 = sum(score["rouge2"].fmeasure for score in rouge_scores) / len(rouge_scores)
    avg_rougeL = sum(score["rougeL"].fmeasure for score in rouge_scores) / len(rouge_scores)

    return {
        "bleu": avg_bleu,
        "meteor": avg_meteor,
        "rouge1": avg_rouge1,
        "rouge2": avg_rouge2,
        "rougeL": avg_rougeL,
    }

# Creazione Seq2SeqTrainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

# Avvia il training
trainer.train()

# Salvataggio finale LoRA + quantization
trainer.save_model("./results/best_model")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# prompt: zip the results and upload it on drive

import os
import zipfile

# Specify the directory to zip
directory_to_zip = "/content/ifttt-code-generator"  # Replace with the actual directory

# Specify the zip file name
zip_file_name = "/content/drive/MyDrive/ifttt_results.zip"

# Create a zip archive
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(directory_to_zip):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, directory_to_zip))

print(f"Successfully zipped the contents of '{directory_to_zip}' to '{zip_file_name}'")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os
import torch
# Percorso del modello fine-tunato
finetuned_model_path = "./results/best_model"
basemodel_path = "mistralai/Mistral-7B-Instruct-v0.2"
# Caricamento del modello base


#create offload directory if it doesn't exist
if not os.path.exists("./offload"):
    os.makedirs("./offload")


# Caricamento del modello e del tokenizer fine-tunati
print("Caricamento del modello fine-tunato...")
from peft import PeftModel

bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(
    basemodel_path,
    torch_dtype=torch.float16,         # or torch.bfloat16, depending on your setup
    quantization_config=bnb_config,           # 4-bit quantization
    device_map="auto",
    offload_folder="./offload"         # <= Provide a folder path
)

model = PeftModel.from_pretrained(
    model,
    finetuned_model_path,
)


tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)


In [ ]:

# Funzione per generare il codice IFTTT
def generate_ifttt_code(prompt, max_length=512, num_return_sequences=1):
    # Tokenizzazione dell'input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generazione del codice
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
    )

    # Decodifica del risultato
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded_outputs

# Prompt per generare il codice IFTTT
prompt = "if the current hour is 17, send a tweet"
generated_code = generate_ifttt_code(prompt)

# Stampa del codice generato
print("\nCodice IFTTT generato:")
for i, code in enumerate(generated_code, 1):
    print(f"{code}")
